In [60]:
from pyowm.owm import OWM
from pyowm.utils import timestamps
from dotenv import load_dotenv
from pydantic import Field
import os
import requests
from datetime import datetime, timezone, timedelta

In [1]:
import openmeteo_requests

import requests_cache
import pandas as pd
import numpy as np
from retry_requests import retry



# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://historical-forecast-api.open-meteo.com/v1/forecast"
params = {
	"latitude": 52.52,
	"longitude": 13.41,
	"start_date": "2024-01-01",
	"end_date": "2024-12-31",
	"daily": ["temperature_2m_max", "temperature_2m_min"]
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}

daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min

daily_dataframe = pd.DataFrame(data = daily_data)
daily_dataframe

,date,temperature_2m_max,temperature_2m_min
0,2024-01-01 00:00:00+00:00,7.795500,4.8455
1,2024-01-02 00:00:00+00:00,7.595500,2.7955
2,2024-01-03 00:00:00+00:00,11.045500,7.8955
3,2024-01-04 00:00:00+00:00,8.445499,0.2955
4,2024-01-05 00:00:00+00:00,0.895500,0.1455
...,...,...,...
361,2024-12-27 00:00:00+00:00,4.545500,-0.0045
362,2024-12-28 00:00:00+00:00,-0.004500,-2.1045
363,2024-12-29 00:00:00+00:00,1.245500,-3.3545
364,2024-12-30 00:00:00+00:00,3.045500,0.1955


In [ ]:
def get_weather_data(
    lat: float = Field(
        ..., description = "The latitude of the destination city"
    ), 
    lon: float = Field(
        ..., description = "The longtitude of the destination city"
    )
):
    
    cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)
    
    # Make sure all required weather variables are listed here
    # The order of variables in hourly or daily is important to assign them correctly below
    url = "https://historical-forecast-api.open-meteo.com/v1/forecast"
    params = {
    	"latitude": lat,
    	"longitude": lon,
    	"start_date": "2024-01-01",
    	"end_date": "2024-12-31",
    	"daily": ["temperature_2m_max", "temperature_2m_min"]
    }
    responses = openmeteo.weather_api(url, params=params)
    
    # Process first location. Add a for-loop for multiple locations or weather models
    response = responses[0]
    
    # Process daily data. The order of variables needs to be the same as requested.
    daily = response.Daily()
    daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
    daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
    
    daily_data = {"date": pd.date_range(
    	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
    	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
    	freq = pd.Timedelta(seconds = daily.Interval()),
    	inclusive = "left"
    )}
    
    daily_data["temperature_2m_max"] = daily_temperature_2m_max
    daily_data["temperature_2m_min"] = daily_temperature_2m_min
    
    daily_dataframe = pd.DataFrame(data = daily_data)
    return daily_dataframe

In [ ]:
def get_geolocation(
    destination: str = Field(
        ..., description = "The city which the latitude and longtitude of will be fetched"
    )
) -> dict:
    
    url = "https://nominatim.openstreetmap.org/search"
    params = {"q": destination, "format":"json"}  # Dictionary of parameters
    headers = {"Accept": "application/json", "Referer": "https://nominatim.openstreetmap.org/"}
    response = requests.get(url, params=params, headers=headers)

    result = {}
    if response.status_code == 200:
        data = response.json()
        result["lat"] = float(data[0]["lat"])
        result["lon"] = float(data[0]["lon"])
        return result
    else:
        print(f"Error: {response.status_code}")